In [14]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import random
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [15]:
n_train = 6000
n_test = 1000
batch_size = 256

In [16]:
def get_batch_jacobian(net, x, target):
    net.zero_grad()
    x.requires_grad_(True)
    y = net(x)
    
    y.backward(torch.ones_like(y))
    jacobian = x.grad.detach()
    return jacobian, target.detach()

In [17]:
def eval_score(jacob):
    correlations = np.corrcoef(jacob)
    v, _ = np.linalg.eig(correlations)
    k = 1e-5
    return -np.sum(np.log(v + k) + 1./(v + k))

In [18]:
def load_dataset(n_train, n_test, batch_size):
    """
    Loads train & test sets from MNIST with user-specified sizes.

    Args:
        n_train (int): Desired number of samples in the training set.
        n_test (int): Desired number of samples in the testing set.
        batch_size (int): Batch size for the DataLoaders.

    Returns:
        tuple: (train_loader, test_loader) where each loader is a
               torch.utils.data.DataLoader.
    """
    # Define transformations for the dataset
    transform = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.1307,), (0.1381,)),
                                    transforms.Lambda(lambda img: F.interpolate(img.unsqueeze(0), size=(14, 14), 
                                        mode='bilinear', align_corners=False).squeeze(0))])

    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    # Subset the datasets to the desired number of samples
    train_subset = torch.utils.data.Subset(train_dataset, range(n_train))
    test_subset = torch.utils.data.Subset(test_dataset, range(n_test))

    # Create DataLoaders for training and testing sets
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

    print("Number of training samples:", len(train_subset))
    print("Number of test samples:", len(test_subset))

    return train_loader, test_loader

train_loader, test_loader = load_dataset(n_train, n_test, batch_size)

Number of training samples: 6000
Number of test samples: 1000


In [19]:
def extract_image_patches(x):
    # Do TF 'SAME' Padding
    B, C, H, W = x.shape  
    #x = torch.arange(B*C*H*W).view(B, C, H, W)
    kernel_h, kernel_w = 2, 2
    stride = 2

    patches = x.unfold(2, kernel_h, stride).unfold(3, kernel_w, stride)
      
    patches = patches.contiguous().view(B, H // stride, W // stride, -1)

    return patches.reshape(-1, 4)

In [20]:
def get_random_gate():
    gate = np.random.randint(1, 5) # 1..4
    wire1 = np.random.randint(0, 4) # 0..3
    wire2 = np.random.randint(0, 4) # 0..3
    if gate == 3:
        while wire1 == wire2:
            wire2 = np.random.randint(0, 4)
              
    return 100 * gate + 10 * wire1 + wire2

def generate_circuit(n_gates):
    circuit = []
    n_weights = 0
    fitness = 0
    for _ in range(n_gates):
        gate = get_random_gate()
        if (gate // 100) % 10 in [1, 2]:
            n_weights += 1
        circuit.append(gate)
    return (circuit, fitness, n_weights) 


def encode_gate(gate):
    if isinstance(gate, qml.RX):
        return 100 + gate.wires[0] * 10 
    elif isinstance(gate, qml.RZ):
        return 200 + gate.wires[0] * 10 
    elif isinstance(gate, qml.CNOT):
        return 300 + gate.wires[0] * 10 + gate.wires[1] * 1 
    elif isinstance(gate, qml.Hadamard):
        return 400 + gate.wires[0] * 10 
    else:
        print(gate)
        raise Exception("Invalid gate")
    
def decode_gate(encoded_gate, param):
    """
    D D D
        - Gate   [1..4]
        - Wire 1 [0..N_QUBITS-1]
        - Wire 2 [0..N_QUBITS-1]
    """
    encoded_gate = int(encoded_gate.item()) # Convert tensor to int to be able to use adjoint differentiation
    wire2  = encoded_gate % 10
    wire1 = (encoded_gate // 10) % 10
    gate = (encoded_gate // 100) % 10
    
    if gate == 1:
        return qml.RX(param, wires=wire1)
    elif gate == 2:
        return qml.RZ(param, wires=wire1)
    elif gate == 3:
        return qml.CNOT(wires=[wire1, wire2])
    elif gate == 4:
        return qml.Hadamard(wires=wire1)
    else:
        print(encoded_gate)
        raise Exception("Invalid gate")

In [21]:
out_dims_xy = 7

dev = qml.device("lightning.qubit", wires=4)

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def circuit(inputs, weights):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * inputs[j], wires=j)
    
    qml.RX(weights[0], wires=0)
    qml.RX(weights[1], wires=1)

    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])
    
    qml.RY(weights[2], wires=0)
    qml.RY(weights[3], wires=3)

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

qlayer = qml.qnn.TorchLayer(circuit, {"weights": (4,)})
dim = 14 // 2
class HQNN(nn.Module):
    def __init__(self):
        super(HQNN, self).__init__()

        # Quanvolutional layer (Maps 1 input channel to 4 output channels)
        self.quanv = qlayer

        # Fully connected layer to perform the final classification
        self.fc1 = nn.Linear(dim * dim * 4, 1)  # Assuming 10 output classes

    def forward(self, input):
        patches = extract_image_patches(input)  

        
        quanvoluted_patches = torch.stack([self.quanv(patch) for patch in patches])
        x = quanvoluted_patches.reshape(input.shape[0], dim, dim, 4)

        x = x.view(-1, dim * dim * 4)  # Flatten for the fully connected layer
        x = self.fc1(x)
        return x

In [22]:
net = HQNN()


data_iterators = iter(train_loader)
x, target = next(data_iterators)

jacobian, labels = get_batch_jacobian(net, x, target)
jacobian = jacobian.reshape(batch_size, -1)

try:
    s = eval_score(jacobian)
except Exception as e:
    print(e)
    s = np.nan
    
print(s)

Array must not contain infs or NaNs
nan


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


In [23]:
dev = qml.device('lightning.qubit', wires=4)

@qml.qnode(dev,interface="torch", diff_method="adjoint")
def circuit(inputs, weights):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * inputs[j], wires=j)

    for i in range (2):
        qml.CNOT(wires=[0, 2])
        qml.CNOT(wires=[1, 3])
        qml.RZ(weights[0+i], wires=3 -i)
        qml.RX(weights[1 + i], wires=1- i)
        qml.CNOT(wires=[3-i, 2- i])
        qml.CNOT(wires=[1, 0])
        qml.RZ(weights[2+ i], wires=1-i)
        qml.Hadamard(wires=3-i)

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

In [24]:
qlayer = qml.qnn.TorchLayer(circuit, {"weights": (6,)})
dim = 14 // 2
class HQNN(nn.Module):
    def __init__(self):
        super(HQNN, self).__init__()

        # Quanvolutional layer (Maps 1 input channel to 4 output channels)
        self.quanv = qlayer

        # Fully connected layer to perform the final classification
        self.fc1 = nn.Linear(dim * dim * 4, 1)  # Assuming 10 output classes

    def forward(self, input):
        patches = extract_image_patches(input)  

        
        quanvoluted_patches = torch.stack([self.quanv(patch) for patch in patches])
        x = quanvoluted_patches.reshape(input.shape[0], dim, dim, 4)

        x = x.view(-1, dim * dim * 4)  # Flatten for the fully connected layer
        x = self.fc1(x)
        #x = F.relu(x)  
        return x

In [25]:
net = HQNN()


data_iterators = iter(train_loader)
x, target = next(data_iterators)

jacobian, labels = get_batch_jacobian(net, x, target)
jacobian = jacobian.reshape(batch_size, -1)

try:
    s2 = eval_score(jacobian)
except Exception as e:
    print(e)
    s2 = np.nan
    
print(s2)

(-8435912.095733166-6.776263578034403e-21j)


In [26]:
print (s2 > s)

False
